In [17]:
import math
import pandas as pd
import numpy as np
from collections import deque
from itertools import combinations

def binary_permutations(lst: []) -> [[]]:
    result = []
    for comb in combinations(range(len(lst)), lst.count(1)):
        permutation = [0] * len(lst)
        for i in comb:
            permutation[i] = 1
        result.append(permutation)
    return result

def calculate_z(parity_vector: []) -> int:
    indices = []
    for i, entry in enumerate(parity_vector, start=0):
        if entry == 1:
            indices.append(i)
    
    result = 0
    U = len(indices)
    for i, entry in enumerate(indices, start=0):
        result = result + 3**(U-(i+1)) * 2**(entry)
    return result

def generate_df(n: int, U: int) -> pd.DataFrame:
    if n == 0 or U == 0 or n < U:
        return None

    df = pd.DataFrame(columns = ['parity_vector', 'shifted orbits', 'z vals', 'min_z'])
    p_v = [0] * n
    for i in range(U):
        print(i)
        print(U)
        p_v[i] = 1
    permutations = binary_permutations(p_v)
    for i in range(len(permutations)):
        vec = permutations[i]
        z_vals = []
        shifted_vecs = []
        shifted = vec.copy()

        for j in range(len(shifted)):
            shifted_vecs.append(''.join(map(str, shifted)))
            z_vals.append(calculate_z(shifted))
            temp = deque(shifted)
            temp.rotate(-1)
            shifted = list(temp)
        min_z_vals = min(z_vals) if len(z_vals) > 0 else 0
        df.loc[i] = [''.join(map(str, vec)), shifted_vecs, z_vals, min_z_vals]
    return df

def calculate_N(M:int, n:int, U:int) -> int:
    r = math.gcd(n, U)
    sum = 0
    for i in range(r):
        sum += 2**(i*n/r)*3**(U-1-i*U/r)
    return 2*M - int(sum)

#n=14
#U=8
#df = generate_df(n, U)
#M = df['min_z'].max()
#N = calculate_N(M, n, U)
#c = 2**n-3**U
#print("M({0},{1})={2}".format(n, U, M))
#print("N({0},{1})={2}".format(n, U, N))
#print("c={0}".format(c))
#
#df.head(4)


In [18]:
max_n=5
data = np.empty((0, 4), int)
for x in range(max_n):
    data_n = np.empty((0, 4), int)
    for y in range(max_n):
        df = generate_df(x, y)
        M = 0 if df is None else df['min_z'].max()
        N = 0 if M == 0 else calculate_N(M, x, y)
        data_n = np.append(data_n, np.array([[x, y, M, N]]), axis=0)
    print("x : {0}, y: {1}".format(x,y))
    data=np.append(data, data_n, axis=0)

columns = ['x', 'y', 'M', 'N']
df = pd.DataFrame(data, columns=columns)
df.to_csv('c:/temp/collatz_M_N.csv', index=False)

x : 0, y: 4
0
1
0
2
1
2


IndexError: list assignment index out of range